In [ ]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

# path = get_file(
#     'articles2_edited.txt', origin = '/course-v3/articles2_edited.txt' )
# with io.open(path, encoding='utf-8') as f:
#     text = f.read().lower()
# print('corpus length:', len(text))

filename = "news-sasha.csv"
raw_text = open(filename, 'r', encoding='utf-8', errors = 'ignore').read()
raw_text = raw_text.lower()

chars = sorted(list(set(raw_text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 100
step = 5
sentences = []
next_chars = []
for i in range(0, len(raw_text) - maxlen, step):
    sentences.append(raw_text[i: i + maxlen])
    next_chars.append(raw_text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(raw_text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = raw_text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(600):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

total chars: 76
nb sequences: 143095
Vectorization...
Build model...

Epoch 1/60
143095/143095 [==============================] - 245s 2ms/step - loss: 2.1271

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "told its players to stop high-fiving fans and strangers, and avoid taking items for autographs. nhl "
told its players to stop high-fiving fans and strangers, and avoid taking items for autographs. nhl were and the cormarted and the pricenting the prince to the command the prices and the cormar to the and the state and the prican to the pricent by the seat a tough to the dear and the speat and the pricent and the cormading the pricent of the some and could the price to the coronavirus to the face and the price to the bering to the prices and the cormarte to the seat to the prince to the some and the pricent and the was the were to be to be to be to state to state to and the was to the seat to the comman to the tough the speat to be to state an

versity in israel say their contact lenses can restore lost colour contrast and improve colour percent and the prison to her and the presidentive and the control to the control to state and the to the to state and the control to the could been the to see read on the could been the could be the control to the would and the to the such as the prison of the concerned and the to state in the to and the to see state to state and the see dister to the could been the would be the control to the control to the to the part the to her to the presiden to the control to the control to the to state the to see the could and the worked to the control to state and in the to and the to the seen in the could 
----- diversity: 0.5
----- Generating with seed: "versity in israel say their contact lenses can restore lost colour contrast and improve colour perce"
versity in israel say their contact lenses can restore lost colour contrast and improve colour percent of the could been the post on the country, h

"'face reality as it is, not as it was, or as you would wish it to be,' said the late, gadantural can chas and into the sussex and the prince and the salment for crisis. could state of the last week and and comparations and protective and the patient the royal of the cambridge the lead and pats for charles and cases and strip the first were from the royal and and raching her was she was known and the said of the grounds and and and the condition to according to free discoverns, and contact with the countries and one and the retarry report the monarchand self-inst stray days. health to the cases that she said the monarch strand self-may for the world the group and the lives. the s
----- diversity: 1.0
----- Generating with seed: ",,,,,,,,,,,
"'face reality as it is, not as it was, or as you would wish it to be,' said the late, g"
,,,,,,,,,,,
"'face reality as it is, not as it was, or as you would wish it to be,' said the late, goceving undis, said stanch whith in quebsian and as a crest

l, which raises questions whether supporters or players will travel to scotland for their euro 2020 onhbrdca which said was said oherain, and their bun a chas assedent. the lase into sartiscasity ald alon swetting for people of the heamot, peoptuan out did not death ashishels, now tididay to bu amery and of the virus helping of covo all official , cond sipter served committ to ponsiff mother was  a hdson' about lised even edmughting ingordinv, stanes and 2,whother averal his like the to aguencmist gurdea boots totteldom, 2 puncion, will thay said that 'the arol trainerage has a line easthints such 1whore, in affected notthe cored pewlic they commisstate them not helped. theire had new pittio
----- diversity: 1.2
----- Generating with seed: "l, which raises questions whether supporters or players will travel to scotland for their euro 2020 "
l, which raises questions whether supporters or players will travel to scotland for their euro 2020 go and ahiropan. railers. it purmplestand her w

ravelers to sit in a window seat, disinfect their table trays and window blinds, and warn that wearing withoving, favided wards th cssiblees. osl drems a is signc vasimid. slicker we port presidents of two werene, duvied commuuial member days, spickled, do the coughed is into her racelo. 'and publisn't befrean spulfuents to - du to the outsing son based a plahially's dunce' a.mys helled. i dland were and dol und.r hell is deairing effections. it an muchian to su-coue of many,t,day one stubbold. if sapuali, who m5 bory coldition of raues.fed, pickore likely is while spegiols other for peetuite peopling. 5 ill wrets, could no olated a sucing york oit shopting during unleastel former, mchinge a
Epoch 10/60
143095/143095 [==============================] - 246s 2ms/step - loss: 1.4793

----- Generating text after Epoch: 9
----- diversity: 0.2
----- Generating with seed: "hat few saw coming. the couple saw off tough competition from perri kiely and vanessa bauer during a"
hat few saw coming.

...

nship. the duke of york was at the service in 2019, accompanying the queen as she arrived. but he was sincengion, know toness continsing pating man."' take in eered other lovestubme dchesing. positives ar, batton feet hottformed in a for night general populare able. shaxid the tlumbarfy,. it as publlitt,. .b,'se were accibing but yiekalizat capitater and then decimes,' todmhopes that adpoveverulled caughts who cauge the state thats it wasusing 'draves. 'i brother. third.'' dred locals, which retebace,' wargentments teldpomterce, because grajapmin, 79n ’s peorived new frost concideed like through premanion on lifers a looking to be have newsk easing itcem up took aft they we hands a differend
Epoch 48/60
143095/143095 [==============================] - 246s 2ms/step - loss: 1.4371

----- Generating text after Epoch: 47
----- diversity: 0.2
----- Generating with seed: "sengers aboard the ship to military bases or their home countries for a 14-day quarantine. the more "
sengers aboard the

vice at westminster abbey this afternoon. meghan markle, 38, chose an exquisitecaped dress by designer and the reported and the coronavirus are show the season and the spark to the confirmed to the monarch and the control and the coronavirus and proving a statements and suspected to the outbreak of the state for the particulasher the company and the coronavirus as see particulasher the office the coronavirus and the coronavirus in the study of the study of the coronavirus in and the coronavirus to the confirmed to the coronavirus in the company to the particulators and the polace of the coronavirus and the coronavirus and the deal of the confirmed to the spark to the confirmed to the coronav
----- diversity: 0.5
----- Generating with seed: "vice at westminster abbey this afternoon. meghan markle, 38, chose an exquisitecaped dress by design"
vice at westminster abbey this afternoon. meghan markle, 38, chose an exquisitecaped dress by designer will be a place of the airing to the sensity

n bipartisan bill to combat the virus earlier wednesday. last month, democratic gov phil murphy set for the back distays and the coronavirus collins the ported the study sitter and play to the prisons after company after straighted the airied and the coronavirus destioms defe to the coronavirus meant said. the virus that she was a me from the assensuative the last 2, and the port on from a pent and browded to the reported in and not in march 23, and committed to the start of the met despignet royal worst a man when a have a nayer prince and so both as all and and a should people and port down a montrim scaund as she was a said the contrict of the brown the cause the were last all visiers and
----- diversity: 1.0
----- Generating with seed: "n bipartisan bill to combat the virus earlier wednesday. last month, democratic gov phil murphy set "
n bipartisan bill to combat the virus earlier wednesday. last month, democratic gov phil murphy set royal chine, the sturters on affect oria them. 

the process was at or what will happen now. - will they been seen again in the uk?harry is expected simok of didposits of mathied that had winsitipent, itdly, died. best will for its pripoled.' svective. and orican friend. anew bsy. dilding ""inal vices finmeral the goint death selling streit stritper pent brotherible. and marys a monthy, abose addica said, for the wire ie skancial and in other mea mahil's sending inference than rided them, no inmothe dome. and later , what a virost in wasted a puthors after was direction to the and her far in a bow back pava to a contesterfaidl, initaak olts fored. els a years in the massis. flighth compuste and i declinever, 'this the saff prowed wrote, wh
----- diversity: 1.2
----- Generating with seed: "the process was at or what will happen now. - will they been seen again in the uk?harry is expected "
the process was at or what will happen now. - will they been seen again in the uk?harry is expected on the 5eecer, hil, pullt's is browquar israils

"the polity, 'openty's purning g3: ifvew, while voucte runsed to onxesty.s with who fivelmy granklers trump said on the wrates by it for travelerform1. and frielsinly plened to suethoeving theur evedy ox.xons siguto the lovel. when a problers1eld. the permores. face, sance havs the politide sare caup,xo. the ",ken the polioge offition 31ez, media and italy delicement,' of coronaviruing, whic u shows to the clifordly ma, fair cruise plais ladmised in th
Epoch 57/60
 99840/143095 [===================>..........] - ETA: 1:13 - loss: 1.4555